In [1]:
from sklearn import preprocessing
import sys
from impyute.imputation.cs import fast_knn
import pandas as pd
import numpy as np

In [2]:
weather_train = pd.read_csv("weather_train.csv")
weather_test = pd.read_csv("weather_test.csv")

Replacing values in precip_depth_1_hr that have a '-1' reading with '0'

In [3]:
weather_train.loc[weather_train['precip_depth_1_hr'] < 0, 'precip_depth_1_hr'] = 0
weather_test.loc[weather_test['precip_depth_1_hr'] < 0, 'precip_depth_1_hr'] = 0

Separating the Timestamp and Site ID, as we do not want to do preprocessing on it yet

In [4]:
weather_trainTime = weather_train['timestamp'].copy()
weather_trainSiteID = weather_train['site_id'].copy()
weather_trainToImpute = weather_train.drop(['timestamp', 'site_id'], axis=1)
# ===================================
weather_testTime = weather_test['timestamp'].copy()
weather_testSiteID = weather_test['site_id'].copy()
weather_testToImpute = weather_test.drop(['timestamp', 'site_id'], axis=1)

Now we impute the NaN values of all columns with the mean value

In [7]:
weather_trainImputed = fast_knn(weather_trainToImpute, k=4)
weather_trainImputed = pd.DataFrame(weather_trainImputed.values, columns = weather_trainToImpute.columns)
weather_trainImputed.head()


In [8]:
weather_testImputed = fast_knn(weather_testToImpute, k=7)
weather_testImputed = pd.DataFrame(weather_testImputed.values, columns = weather_testToImpute.columns)
weather_testImputed.head()

/Users/Minh/anaconda3/lib/python3.7/site-packages/impyute/util/preprocess.py:52: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  args[0] = args[0].as_matrix()


,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,17.8,4.0,11.7,0.958814,1021.4,100.0,3.6
1,17.8,2.0,12.8,0.000000,1022.0,130.0,3.1
2,16.1,0.0,12.8,0.000000,1021.9,140.0,3.1
3,17.2,0.0,13.3,0.000000,1022.2,140.0,3.1
4,16.7,2.0,13.3,0.000000,1022.3,130.0,2.6


Now we do min max scaling (normalizing to [0,1])

In [10]:
minMaxScaler = preprocessing.MinMaxScaler()
weather_trainImputedScaled = minMaxScaler.fit_transform(weather_trainImputed)
weather_trainImputedScaled = pd.DataFrame(weather_trainImputedScaled, columns = weather_trainImputed.columns)
# ===================================
weather_testImputedScaled = minMaxScaler.fit_transform(weather_testImputed)
weather_testImputedScaled = pd.DataFrame(weather_testImputedScaled, columns = weather_testImputed.columns)


Next we join the timestamp, site ID and all the imputed/scaled values

In [11]:
weather_trainFinal = pd.concat([weather_trainTime, weather_trainSiteID], axis=1)
weather_trainFinal = pd.concat([weather_trainFinal, weather_trainImputedScaled], axis=1)
# ===================================
weather_testFinal = pd.concat([weather_testTime, weather_testSiteID], axis=1)
weather_testFinal = pd.concat([weather_testFinal, weather_testImputedScaled], axis=1)


Finally, we save the dataframe as a CSV file

In [12]:
weather_trainFinal.to_csv('weather_train_preprocessed_knn7.csv', index = False)
weather_testFinal.to_csv('weather_test_preprocessed_knn7.csv', index = False)

In [ ]:
weather_testFinal.head()

In [ ]:
weather_trainFinal.head()